<a href="https://colab.research.google.com/github/inwidyana/final_project_machine_learning/blob/master/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.models import Model
from keras.layers import Input, GRU, Dense
import numpy as np
import tensorflow as tf
import os

Using TensorFlow backend.


In [0]:
batch_size = 64  # Batch size for training.
epochs = 1  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 100  # Number of samples to train on. My memory is killing me

path_to_zip = tf.keras.utils.get_file(
    'training-parallel-nc-v8.tgz', origin='http://statmt.org/wmt13/training-parallel-nc-v8.tgz',
    extract=True)

# Dataset
data_de = os.path.dirname(path_to_zip)+"/training/news-commentary-v8.de-en.de"
data_en = os.path.dirname(path_to_zip)+"/training/news-commentary-v8.de-en.en"

In [3]:
# Set contains unique values
input_characters = set()
target_characters = set()

# Split each line to lists
with open(data_de, 'r', encoding='utf-8') as data:
    input_texts = data.read().split('\n')
with open(data_en, 'r', encoding='utf-8') as data:
    target_texts = data.read().split('\n')

input_texts = input_texts[: min(num_samples, len(input_texts) - 1)]
target_texts = target_texts[: min(num_samples, len(target_texts) - 1)]

# Add new chars
for line in input_texts:
    for char in line:
        if char not in input_characters:
            input_characters.add(char)
for line in target_texts:
    for char in line:
        if char not in target_characters:
            target_characters.add(char)

# Sort
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

# How many different characters
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

# Sequence max lengths
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of training data :', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max input length:', max_encoder_seq_length)
print('Max output length:', max_decoder_seq_length)

# Map a number to the characters
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

# (size of input, max sequence, number of unique possible values)
encoder_input = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens))
decoder_input = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens))
decoder_target = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens))

# zip returns set of index + item
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep and will not include the start character.
            decoder_target[i, t - 1, target_token_index[char]] = 1.

Number of training data : 100
Number of unique input tokens: 73
Number of unique output tokens: 71
Max input length: 320
Max output length: 284


In [4]:
# Initiate Keras tensor input
encoder_inputs = Input(shape=(None, num_encoder_tokens))
# Return state = output of the last time step , return sequence = returns all time step
encoder = GRU(latent_dim, return_state=True)
encoder_outputs, state_h = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states. h = hidden

W0622 13:10:00.072146 140061441275776 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0622 13:10:00.093815 140061441275776 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0622 13:10:00.099984 140061441275776 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [5]:
# Set up the decoder, using encoder_states as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_gru = GRU(latent_dim, return_sequences=True)
decoder_outputs = decoder_gru(decoder_inputs,
                                     initial_state=state_h)

# Softmax layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model architecture
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Train
model.compile(optimizer ='rmsprop', loss ='categorical_crossentropy', metrics = ['accuracy'])
model.fit([encoder_input, decoder_input], decoder_target,
          batch_size=batch_size,
          epochs=epochs,
validation_split=0.2)

W0622 13:10:11.892312 140061441275776 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0622 13:10:11.913230 140061441275776 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0622 13:10:12.037016 140061441275776 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0622 13:10:13.104914 140061441275776 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated

Train on 80 samples, validate on 20 samples
Epoch 1/1
80/80 [==============================] - 6s 72ms/step - loss: 1.5828 - acc: 0.5542 - val_loss: 1.4677 - val_acc: 0.7018
